In [1]:
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    import pickle
    from sklearn.metrics import confusion_matrix, accuracy_score,average_precision_score,classification_report,f1_score
    from sklearn.compose import ColumnTransformer
    import matplotlib.pyplot as plt
    from sklearn.ensemble import RandomForestClassifier
    import os
    df_raw = pd.read_csv(r'dataset/Train.tsv', sep='\t',dtype=object)
    df = df_raw.copy()
    print(df.columns)
    df.columns=['Payer', 'Debit_CreditInd.', 'PercentofPayAmount', 'DaystoPay']
    df.drop(df[df['Payer']=='*'].index,inplace=True)
    df.drop(columns='PercentofPayAmount',inplace=True)
    Convert_dict = {'Payer': str,
                'Debit_CreditInd.': str,
                'DaystoPay': float
               }
    df = df.astype(Convert_dict)
    print(df.head())
    print(df.describe())
    print(df.isnull().sum())
    dataseti=df.groupby(['Payer','Debit_CreditInd.'])['DaystoPay'].mean()
    datasetii = pd.DataFrame(dataseti).reset_index()
    inner_join = pd.merge(df,
                      datasetii,
                      on =['Payer','Debit_CreditInd.'],
                      how ='inner')
    inner_join['rank'] = inner_join.groupby(['Payer','Debit_CreditInd.'])['DaystoPay_x'].rank()
    dataset_ready=inner_join[['Payer','Debit_CreditInd.','rank','DaystoPay_x']]
    dataset=dataset_ready
    X1=dataset[['Payer','Debit_CreditInd.','rank']]

    #OnehotEncoding
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False,handle_unknown='ignore'), [0,1])], remainder='passthrough')
    X = np.array(ct.fit_transform(X1))
    pickle.dump(ct,open('model_files/01ohe.sav', 'wb'))

    y = dataset.iloc[:, -1].values
    #Training Model
    classifier = RandomForestClassifier(n_estimators = 75, criterion = 'entropy', random_state = 0,bootstrap=True,
                                    ccp_alpha=0.0, class_weight=None, max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0,n_jobs=None, oob_score=False, verbose=0,
                       warm_start=False)
    classifier.fit(X, y)
    pickle.dump(classifier,  open('model_files/02randomforest.sav', 'wb'))

    #Score
    y_pred = classifier.predict(X)
    y_prob = classifier.predict_proba(X)
    y_conf = [(np.max(pred_err)*100) for pred_err in y_prob]
    print(y_conf)
    cm = confusion_matrix(y, y_pred)
    #cm = confusion_matrix(y, classifier.predict(X))
    print( accuracy_score(y, y_pred))
    print(classification_report(y, y_pred))
    return(accuracy_score)

Index(['Payer', 'Debit_CreditInd.', 'PercentofPayAmount', 'DaystoPay'], dtype='object')
        Payer Debit_CreditInd.  DaystoPay
134  10000100                S       14.0
135  10000100                S       21.0
136    100030                H        7.0
137    100030                S       47.0
138    100030                S       46.0
          DaystoPay
count  88857.000000
mean      -1.417817
std      931.218377
min   -43902.000000
25%       11.000000
50%       21.000000
75%       25.000000
max      778.000000
Payer               0
Debit_CreditInd.    0
DaystoPay           0
dtype: int64
[65.33333333333333, 74.66666666666667, 72.0, 60.0, 82.66666666666667, 60.0, 89.33333333333333, 96.0, 64.0, 96.0, 58.666666666666664, 89.33333333333333, 100.0, 100.0, 82.66666666666667, 100.0, 100.0, 89.33333333333333, 89.33333333333333, 96.0, 96.0, 92.0, 92.0, 65.33333333333333, 61.33333333333333, 98.66666666666667, 62.66666666666667, 73.33333333333333, 82.66666666666667, 98.66666666666667, 98.6666

              precision    recall  f1-score   support

    -43902.0       1.00      1.00      1.00         1
    -43899.0       1.00      1.00      1.00         1
    -43898.0       1.00      1.00      1.00         1
    -43896.0       1.00      1.00      1.00         1
    -43893.0       1.00      1.00      1.00         1
    -43887.0       1.00      1.00      1.00         1
    -43886.0       1.00      1.00      1.00         1
    -43885.0       1.00      1.00      1.00         9
    -43881.0       1.00      1.00      1.00         1
    -43878.0       1.00      1.00      1.00         1
    -43874.0       1.00      1.00      1.00         1
    -43872.0       1.00      1.00      1.00         1
    -43871.0       1.00      1.00      1.00         5
    -43868.0       1.00      1.00      1.00         3
    -43864.0       1.00      1.00      1.00         6
    -43856.0       1.00      1.00      1.00         1
    -43851.0       1.00      1.00      1.00         1
    -43850.0       1.00    

SyntaxError: 'return' outside function (<ipython-input-1-ff0697bf68ce>, line 61)